In [0]:
# Read the stream
df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", "/Volumes/professional/default/superstore/schema")
    .option("cloudFiles.schemaEvolutionMode", "rescue")
    .option("cloudFiles.maxFilesPerTrigger", 1)
    .load("/Volumes/professional/default/superstore/*.csv")
)

# Clean column names: replace spaces and special chars with underscores
for old_name in df.columns:
    new_name = old_name.strip().replace(" ", "_").replace("-", "_")
    if new_name != old_name:
        df = df.withColumnRenamed(old_name, new_name)
from pyspark.sql.functions import current_timestamp
df=df.withColumn("ingestion_time", current_timestamp())

# Write the stream to Delta table with availableNow trigger
(
    df.writeStream
    .trigger(availableNow=True)\
        .option('checkpointLocation', '/Volumes/professional/default/superstore/schema')\
    .toTable("professional.superstore_bronze.gmart_bronze")
)

In [0]:
df=df.na.drop()

In [0]:
display(df)